# Mecanismos de atención y transformadores

Uno de los principales inconvenientes de las redes recurrentes es que todas las palabras en una secuencia tienen el mismo impacto en el resultado. Esto provoca un rendimiento subóptimo con modelos estándar de codificador-decodificador LSTM para tareas de secuencia a secuencia, como el Reconocimiento de Entidades Nombradas y la Traducción Automática. En realidad, palabras específicas en la secuencia de entrada a menudo tienen más impacto en las salidas secuenciales que otras.

Consideremos un modelo de secuencia a secuencia, como la traducción automática. Este se implementa mediante dos redes recurrentes, donde una red (**codificador**) comprime la secuencia de entrada en un estado oculto, y la otra, el **decodificador**, expande este estado oculto en el resultado traducido. El problema con este enfoque es que el estado final de la red tiene dificultades para recordar el comienzo de una oración, lo que provoca una calidad deficiente del modelo en oraciones largas.

Los **mecanismos de atención** proporcionan un medio para ponderar el impacto contextual de cada vector de entrada en cada predicción de salida de la RNN. La forma en que se implementa es creando atajos entre los estados intermedios de la RNN de entrada y la RNN de salida. De esta manera, al generar el símbolo de salida $y_t$, tomaremos en cuenta todos los estados ocultos de entrada $h_i$, con diferentes coeficientes de peso $\alpha_{t,i}$.

![Imagen que muestra un modelo codificador/decodificador con una capa de atención aditiva](https://drive.google.com/uc?export=view&id=1qJI2_TUn-8nCKbglMR9gadMlJs_ymZoe)
*El modelo codificador-decodificador con mecanismo de atención aditiva en [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citado de [este blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matriz de atención $\{\alpha_{i,j}\}$ representa el grado en que ciertas palabras de entrada influyen en la generación de una palabra determinada en la secuencia de salida. A continuación se muestra un ejemplo de tal matriz:

![Imagen que muestra un alineamiento encontrado por RNNsearch-50, tomado de Bahdanau - arviz.org](https://drive.google.com/uc?export=view&id=1yq9gknxg8abb86zg5EAtOd5Ih2RhDFEU)

*Figura tomada de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Los mecanismos de atención son responsables de gran parte del estado del arte actual o cercano en el procesamiento de lenguaje natural. Sin embargo, agregar atención aumenta enormemente el número de parámetros del modelo, lo que llevó a problemas de escalabilidad con las RNNs. Una restricción clave para escalar las RNNs es que la naturaleza recurrente de los modelos dificulta la paralelización del entrenamiento. En una RNN, cada elemento de una secuencia necesita procesarse en orden secuencial, lo que significa que no puede paralelizarse fácilmente.

La adopción de mecanismos de atención, combinada con esta limitación, condujo a la creación de los modelos transformadores, que ahora representan el estado del arte, como los conocidos BERT y OpenGPT3.

## Modelos transformadores

En lugar de transmitir el contexto de cada predicción previa al siguiente paso de evaluación, los **modelos transformadores** usan **codificaciones posicionales** y atención para capturar el contexto de una entrada determinada dentro de una ventana de texto proporcionada. La imagen a continuación muestra cómo las codificaciones posicionales con atención pueden capturar el contexto dentro de una ventana determinada.

![GIF animado que muestra cómo se realizan las evaluaciones en los modelos transformadores.](https://drive.google.com/uc?export=view&id=16FCJztNJdzvdNm42HxvC68JxOMTQffTm)

Dado que cada posición de entrada se asigna independientemente a cada posición de salida, los transformadores pueden paralelizarse mejor que las RNNs, lo que permite modelos de lenguaje mucho más grandes y expresivos. Cada cabeza de atención puede utilizarse para aprender diferentes relaciones entre palabras que mejoran las tareas de procesamiento de lenguaje natural.

**BERT** (Representaciones de Codificador Bidireccional de Transformadores) es una red de transformadores de múltiples capas muy grande, con 12 capas para *BERT-base*, y 24 para *BERT-large*. El modelo se entrena inicialmente con un gran corpus de datos de texto (Wikipedia + libros) utilizando un entrenamiento no supervisado (prediciendo palabras enmascaradas en una oración). Durante este pre-entrenamiento, el modelo adquiere un nivel significativo de comprensión del lenguaje, que luego puede aprovecharse con otros conjuntos de datos mediante ajuste fino. Este proceso se denomina **aprendizaje por transferencia**.

![Imagen de http://jalammar.github.io/illustrated-bert/](https://drive.google.com/uc?export=view&id=10HQIOlDqnM7deAf-9fhsqPW6U2HnRCKq)

Existen muchas variaciones de arquitecturas de Transformadores, como BERT, DistilBERT, BigBird, OpenGPT3 y más, que pueden ajustarse finamente. El [paquete HuggingFace](https://github.com/huggingface/) proporciona un repositorio para entrenar muchas de estas arquitecturas con PyTorch.

## Usando BERT para clasificación de texto

Veamos cómo podemos usar un modelo BERT pre-entrenado para resolver nuestra tarea tradicional: clasificación de secuencias. Vamos a clasificar nuestro conjunto de datos original de AG News.

Primero, carguemos la biblioteca HuggingFace y nuestro conjunto de datos:


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import evaluate

In [ ]:
ds = load_dataset("fancyzhx/ag_news")
train_ds, test_ds = ds["train"], ds["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
train_ds[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [ ]:
# Check the features of the dataset
print(ds['train'].features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}


Dado que usaremos un modelo BERT preentrenado, necesitaremos usar un tokenizador específico. Primero, cargaremos un tokenizador asociado con el modelo BERT preentrenado.

La biblioteca HuggingFace contiene un repositorio de modelos preentrenados, que puedes usar simplemente especificando sus nombres como argumentos para las funciones `from_pretrained`. Todos los archivos binarios necesarios para el modelo se descargarán automáticamente.

Sin embargo, en ciertos momentos, es posible que necesites cargar tus propios modelos. En ese caso, puedes especificar el directorio que contiene todos los archivos relevantes, incluidos los parámetros para el tokenizador, el archivo `config.json` con los parámetros del modelo, los pesos binarios, etc.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
train_ds_tk = train_ds.map(tokenize_function, batch_size = 32, batched=True)
test_ds_tk = test_ds.map(tokenize_function, batch_size = 32, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
tokenizer.encode('HuggingFace is the greatest framework for NLP')

[101, 20164, 10932, 2271, 7954, 1110, 1103, 4459, 8297, 1111, 21239, 2101, 102]

🤗 Transformers proporciona una clase `Trainer` optimizada para el entrenamiento de modelos de 🤗 Transformers, lo que facilita comenzar el entrenamiento sin tener que escribir manualmente tu propio bucle de entrenamiento. La API de `Trainer` admite una amplia gama de opciones y características de entrenamiento, como registro de métricas, acumulación de gradientes y precisión mixta.

Comienza cargando tu modelo y especifica el número de etiquetas esperadas. Del conjunto de datos `ag_news`, sabemos que hay cuatro etiquetas:


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Verás una advertencia acerca de que algunos de los pesos preentrenados no se están utilizando y que algunos pesos se han inicializado aleatoriamente. ¡No te preocupes, esto es completamente normal! La cabeza preentrenada del modelo BERT se descarta y se reemplaza por una cabeza de clasificación inicializada aleatoriamente. Afinarás esta nueva cabeza del modelo en tu tarea de clasificación de secuencias, transfiriendo el conocimiento del modelo preentrenado a ella.


## Evaluación

`Trainer` no evalúa automáticamente el rendimiento del modelo durante el entrenamiento. Necesitarás pasarle a `Trainer` una función para calcular y reportar métricas. La biblioteca 🤗 Evaluate proporciona una función simple de exactitud (accuracy) que puedes cargar con la función `evaluate.load` (consulta este [quicktour](https://huggingface.co/docs/evaluate/index) para más información):


In [ ]:
metric = evaluate.load("accuracy")

## Entrenamiento del modelo

A continuación, crea una clase `TrainingArguments`, que contiene todos los hiperparámetros que puedes ajustar, así como banderas para activar diferentes opciones de entrenamiento. Para este tutorial, puedes comenzar con los hiperparámetros de entrenamiento predeterminados, pero siéntete libre de experimentar con ellos para encontrar la configuración óptima. Si deseas monitorear tus métricas de evaluación durante el ajuste fino, especifica el parámetro `eval_strategy` en tus argumentos de entrenamiento para informar la métrica de evaluación al final de cada época:



In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

Llama a `compute` en la métrica para calcular la exactitud (accuracy) de tus predicciones. Antes de pasar tus predicciones a `compute`, necesitas convertir los *logits* en predicciones (recuerda que todos los modelos de 🤗 Transformers devuelven *logits*):


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Crea un objeto `Trainer` con tu modelo, los argumentos de entrenamiento, los conjuntos de datos de entrenamiento y prueba, y la función de evaluación:


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tk,
    eval_dataset=test_ds_tk,
    compute_metrics=compute_metrics,
)

Luego, ajusta tu modelo llamando a `train()`:


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
